##### Copyright 2020 The TensorFlow IO Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Lectura de la base de datos PostgreSQL desde TensorFlow IO

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/io/tutorials/postgresql"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/io/tutorials/postgresql.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/io/tutorials/postgresql.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
      <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/io/tutorials/postgresql.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
</table>

## Descripción general

En este tutorial se muestra cómo crear `tf.data.Dataset` desde un servidor de base de datos PostgreSQL, de modo que el `Dataset` creado pueda pasarse a `tf.keras` con fines de entrenamiento o inferencia.

Una base de datos SQL es una fuente importante de datos para los científicos de datos. Como una de las bases de datos SQL de código abierto más populares, [PostgreSQL](https://www.postgresql.org) se usa bastante en las empresas para almacenar datos críticos y transaccionales en todos los ámbitos. Crear `Dataset` directamente desde un servidor de base de datos PostgreSQL y pasar el `Dataset` a `tf.keras` para entrenamiento o inferencia podría simplificar enormemente la canalización de datos y ayudar a los científicos de datos a centrarse en la creación de modelos de aprendizaje automático.

## Configuración y uso

### Instale los paquetes tensorflow-io necesarios y reinicie el tiempo de ejecución

In [ ]:
try:
  %tensorflow_version 2.x
except Exception:
  pass

!pip install tensorflow-io

### Instalar y configurar PostgreSQL (opcional)

**Advertencia: Este bloc de notas fue diseñado para ejecutarse únicamente en Google Colab**. *Instala paquetes en el sistema y requiere acceso de a sudo. Si desea ejecutarlo en un bloc de notas de Jupyter local, hágalo con cuidado.*

Para demostrar el uso en Google Colab, deberá instalar el servidor PostgreSQL. También se necesita la contraseña y una base de datos vacía.

Si no está ejecutando este bloc de notas en Google Colab o prefiere usar una base de datos existente, omita esta configuración y continúe con la siguiente sección.

In [ ]:
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# Setup a database with name `tfio_demo` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS tfio_demo;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE tfio_demo;'

### Configurar las variables del entorno necesarias

Las siguientes variables del entorno se basan en la configuración de PostgreSQL en la última sección. Si tiene una configuración diferente o está usando una base de datos existente, debe cambiarla como corresponde:


In [ ]:
%env TFIO_DEMO_DATABASE_NAME=tfio_demo
%env TFIO_DEMO_DATABASE_HOST=localhost
%env TFIO_DEMO_DATABASE_PORT=5432
%env TFIO_DEMO_DATABASE_USER=postgres
%env TFIO_DEMO_DATABASE_PASS=postgres

### Preparar datos en el servidor PostgreSQL

Para fines de demostración, este tutorial creará una base de datos y la completará con algunos datos. Los datos que se usan en este tutorial provienen del [conjunto de datos de calidad del aire](https://archive.ics.uci.edu/ml/datasets/Air+Quality), disponible en [el repositorio de aprendizaje automático de UCI](http://archive.ics.uci.edu/ml).

A continuación se muestra un adelanto de un subconjunto del conjunto de datos sobre la calidad del aire:

Fecha | Hora | CO(GT) | PT08.S1(CO) | NMHC(GT) | C6H6(GT) | PT08.S2(NMHC) | NOx(GT) | PT08.S3(NOx) | NO2(GT) | PT08.S4(NO2) | PT08.S5(O3) | T | RH | AH
--- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | ---
10/03/2004 | 18.00.00 | 2,6 | 1360 | 150 | 11,9 | 1046 | 166 | 1056 | 113 | 1692 | 1268 | 13,6 | 48,9 | 0,7578
10/03/2004 | 19.00.00 | 2 | 1292 | 112 | 9,4 | 955 | 103 | 1174 | 92 | 1559 | 972 | 13,3 | 47,7 | 0,7255
10/03/2004 | 20.00.00 | 2,2 | 1402 | 88 | 9,0 | 939 | 131 | 1140 | 114 | 1555 | 1074 | 11,9 | 54,0 | 0,7502
10/03/2004 | 21.00.00 | 2,2 | 1376 | 80 | 9,2 | 948 | 172 | 1092 | 122 | 1584 | 1203 | 11,0 | 60,0 | 0,7867
10/03/2004 | 22.00.00 | 1,6 | 1272 | 51 | 6,5 | 836 | 131 | 1205 | 116 | 1490 | 1110 | 11,2 | 59,6 | 0,7888

Puede obtener más información sobre el conjunto de datos de calidad del aire y el repositorio de aprendizaje automático de UCI en la sección de [Referencias](#references).

Para ayudar a simplificar la preparación de datos, se ha preparado una versión SQL del conjunto de datos de calidad del aire y la puede encontrar como [AirQualityUCI.sql](https://github.com/tensorflow/io/blob/master/docs/tutorials/postgresql/AirQualityUCI.sql).

La declaración para crear la tabla es:

```
CREATE TABLE AirQualityUCI (
  Date DATE,
  Time TIME,
  CO REAL,
  PT08S1 INT,
  NMHC REAL,
  C6H6 REAL,
  PT08S2 INT,
  NOx REAL,
  PT08S3 INT,
  NO2 REAL,
  PT08S4 INT,
  PT08S5 INT,
  T REAL,
  RH REAL,
  AH REAL
);
```

Los comandos completos para crear la tabla en la base de datos y completar los datos son:

In [ ]:
!curl -s -OL https://github.com/tensorflow/io/raw/master/docs/tutorials/postgresql/AirQualityUCI.sql

!PGPASSWORD=$TFIO_DEMO_DATABASE_PASS psql -q -h $TFIO_DEMO_DATABASE_HOST -p $TFIO_DEMO_DATABASE_PORT -U $TFIO_DEMO_DATABASE_USER -d $TFIO_DEMO_DATABASE_NAME -f AirQualityUCI.sql

### Crear un conjunto de datos desde el servidor PostgreSQL y usarlo en TensorFlow

Crear un conjunto de datos desde el servidor PostgreSQL es tan fácil como llamar `tfio.experimental.IODataset.from_sql` con argumentos `query` y `endpoint`. La `query` es la consulta SQL para columnas seleccionadas en tablas y el argumento `endpoint` es la dirección y el nombre de la base de datos:

In [ ]:
import os
import tensorflow_io as tfio

endpoint="postgresql://{}:{}@{}?port={}&dbname={}".format(
    os.environ['TFIO_DEMO_DATABASE_USER'],
    os.environ['TFIO_DEMO_DATABASE_PASS'],
    os.environ['TFIO_DEMO_DATABASE_HOST'],
    os.environ['TFIO_DEMO_DATABASE_PORT'],
    os.environ['TFIO_DEMO_DATABASE_NAME'],
)

dataset = tfio.experimental.IODataset.from_sql(
    query="SELECT co, pt08s1 FROM AirQualityUCI;",
    endpoint=endpoint)

print(dataset.element_spec)

Como puede ver en el resultado de `dataset.element_spec` anterior, el elemento del `Dataset` que se crea es un objeto dict de Python con nombres de columnas de la tabla de la base de datos como claves. Es muy conveniente aplicar más operaciones. Por ejemplo, puede seleccionar los campos `nox` y `no2` del `Dataset` y calcular la diferencia:

In [ ]:
dataset = tfio.experimental.IODataset.from_sql(
    query="SELECT nox, no2 FROM AirQualityUCI;",
    endpoint=endpoint)

dataset = dataset.map(lambda e: (e['nox'] - e['no2']))

# check only the first 20 record
dataset = dataset.take(20)

print("NOx - NO2:")
for difference in dataset:
  print(difference.numpy())

El `Dataset` creado ya está listo para pasarse a `tf.keras` directamente para entrenamiento o inferencia.

## Referencias

- Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.
- S. De Vito, E. Massera, M. Piga, L. Martinotto, G. Di Francia, On field calibration of an electronic nose for benzene estimation in an urban pollution monitoring scenario, Sensors and Actuators B: Chemical, Volume 129, Issue 2, 22 February 2008, Pages 750-757, ISSN 0925-4005